# US-Net

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

import matplotlib.pyplot as plt

import cv2

## Camus Dataset

Just to remind myself that \*.raw and \*.mhd files contain the same data.

In [ ]:
import time

@tf.function
def read_raw_image(image_path):
    imgfile = tf.io.read_file(image_path)
    imgbytes = tf.io.decode_raw(imgfile, out_type=tf.uint8)
    img = tf.reshape(imgbytes, (778, 549))
    return img


def read_raw_seq(image_path):
    imgfile = tf.io.read_file(image_path)
    imgbytes = tf.io.decode_raw(imgfile, out_type=tf.uint8)
    img = tf.reshape(imgbytes, (-1, 778, 549))
    return img


def play_sequence(sequence_array):
    cap = cv2.VideoCapture()
    has_frames = True
    frame_counter = 0
    while True:
        frame = sequence_array[frame_counter, :, :]
        cv2.imshow('frame',frame)
        time.sleep(0.1)
        frame_counter+=1
        if frame_counter==sequence_array.shape[0]:
            frame_counter = 0
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
    
img = read_raw_image(r'patient0001/patient0001_4CH_ED.raw')
plt.imshow(img, 'gray')
#play_sequence(seq.numpy())


### Read Sequence

In [ ]:
seq = read_raw_seq(r'patient0001/patient0001_4CH_sequence.raw')
play_sequence(seq.numpy())

Now that we know how to read images and masks, we will process their addresses, and then use tf.data to build the input pipeline. First, we must find the file addresses:

In [ ]:
from glob import glob
from os import path


DATASET_PATH = r'D:\Ultrasound Data\training'


def data_type(filename):
    if filename.endswith('_gt'):
        return 'mask'
    elif filename.endswith('_sequence'):
        return 'sequence'
    else:
        return 'image'

path_structure = path.join(DATASET_PATH, r'*\*.raw')
all_file_paths = glob(path_structure)
train_inputs = []
image_sequences = []

for file_path in all_file_paths:
    filename = path.basename(file_path).split('.')[0]
    if data_type(filename)=='image':
        train_inputs.append(file_path)
    elif data_type(filename)=='sequence':
        image_sequences.append(file_path)

inputs_and_targets = []
for input_file in train_inputs:
    fpath_no_ext = input_file.split('.')[0]
    target_file_path = fpath_no_ext+'_gt.raw'
    if not path.exists(target_file_path):
        print(f'File not found: {target_file_path}')
        continue
    else:
        inputs_and_targets.append([input_file, target_file_path])
assert len(train_inputs) == len(inputs_and_targets)

Now we should convert these file addresses to tensorflow datasets